## Confusion matrix

In [104]:
import pandas as pd
csv_data = pd.read_csv('outputs/run_1/cam1_detections.csv', skiprows=[0], header=None)
y_predictions = csv_data[1].to_numpy()

In [105]:
csv_data[1].shape

(166,)

In [106]:
import numpy as np
y_expected = np.full((166,), 'Luis')

In [107]:
y_predictions, y_expected

(array([' David', 'Luis', 'Luis', 'Luis', 'Luis', 'Luis', 'Luis', 'Luis',
        'Luis', 'Luis', 'Luis', 'Luis', 'Luis', 'Luis', ' Patricio',
        ' Juan', 'Luis', ' David', 'Luis', ' Roberto', ' Julio', ' David',
        ' David', ' Patricio', ' David', ' David', ' David', ' David',
        ' David', 'Luis', 'Luis', ' David', ' Julio', ' Patricio',
        ' David', 'Luis', ' David', ' Juan', 'Luis', ' Patricio', 'Luis',
        'Luis', 'Luis', 'Luis', 'Luis', ' Juan', ' Juan', ' Roberto',
        ' Roberto', ' Juan', ' Roberto', ' Julio', ' David', ' David',
        ' Julio', ' Julio', ' David', ' Julio', ' Julio', ' Roberto',
        ' Julio', ' Julio', ' Juan', ' Patricio', 'Luis', ' Patricio',
        'Luis', 'Luis', 'Luis', ' Patricio', ' Julio', 'Luis', 'Luis',
        ' Roberto', ' Patricio', 'Luis', 'Luis', 'Luis', 'Luis', ' Julio',
        ' Julio', ' Julio', ' Julio', ' Julio', ' Julio', ' Julio',
        ' Julio', ' Julio', ' Julio', ' Roberto', ' Roberto', ' Patricio',

## Doing an array with detection results

In [113]:
y_expected_binary = np.full((166,), True).astype(int)

In [114]:
y_predictions_binary = (y_expected == y_predictions).astype(int)

In [125]:
y_expected_binary[0] = 0

In [121]:
y_predictions_binary

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [120]:
y_expected_binary

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [126]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_expected_binary.astype(int), y_predictions_binary.astype(int))

array([[  2,   0],
       [111,  53]], dtype=int64)

(113, 53)

# Implementing models with Keras

In [1]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

In [2]:
parent_root = 'Datasets/espe/base/test'
target_root = 'Datasets/espe/masked/test'
csv_path = 'Datasets/espe/lbp_histograms/test/test.csv'

from processing import generate_masked_dataset

generate_masked_dataset(parent_root, target_root, csv_path)

Working on:  Datasets/espe/base/test\1
Processing person_1_1.jpg ...
Opening  Datasets/espe/base/test\1\001_9 16.jpg
Saving file on Datasets/espe/masked/test\1\person_1_1.jpg
Processing person_1_2.jpg ...
Opening  Datasets/espe/base/test\1\001_9 17.jpg
Saving file on Datasets/espe/masked/test\1\person_1_2.jpg
Processing person_1_3.jpg ...
Opening  Datasets/espe/base/test\1\001_9 19.jpg
Saving file on Datasets/espe/masked/test\1\person_1_3.jpg
Processing person_1_4.jpg ...
Opening  Datasets/espe/base/test\1\001_9 21.jpg
Saving file on Datasets/espe/masked/test\1\person_1_4.jpg
Processing person_1_5.jpg ...
Opening  Datasets/espe/base/test\1\001_9 22.jpg
Saving file on Datasets/espe/masked/test\1\person_1_5.jpg
Processing person_1_6.jpg ...
Opening  Datasets/espe/base/test\1\001_9 23.jpg
Saving file on Datasets/espe/masked/test\1\person_1_6.jpg
Processing person_1_7.jpg ...
Opening  Datasets/espe/base/test\1\001_9 24.jpg
Saving file on Datasets/espe/masked/test\1\person_1_7.jpg
Processin

In [3]:
parent_root = 'Datasets/espe/base/validation'
target_root = 'Datasets/espe/lbp_histograms - copia/validation'
csv_path = 'Datasets/espe/lbp_histograms - copia/validation/validation.csv' # areas and perimeters csv dataset path
from processing import generate_dataset_with_lbp
generate_dataset_with_lbp(parent_root, target_root, csv_path)

Working on:  Datasets/espe/base/validation\0
Processing person_0_1.jpg ...
Opening  Datasets/espe/base/validation\0\001_9 14.jpg
Saving file on Datasets/espe/lbp_histograms - copia/validation\0\person_0_1.jpg
Processing person_0_2.jpg ...
Opening  Datasets/espe/base/validation\0\001_9 15.jpg
Saving file on Datasets/espe/lbp_histograms - copia/validation\0\person_0_2.jpg
Processing person_0_3.jpg ...
Opening  Datasets/espe/base/validation\0\001_9 18.jpg
Saving file on Datasets/espe/lbp_histograms - copia/validation\0\person_0_3.jpg
Processing person_0_4.jpg ...
Opening  Datasets/espe/base/validation\0\001_9 20.jpg
Saving file on Datasets/espe/lbp_histograms - copia/validation\0\person_0_4.jpg
Processing person_0_5.jpg ...
Opening  Datasets/espe/base/validation\0\001_9 27.jpg
Saving file on Datasets/espe/lbp_histograms - copia/validation\0\person_0_5.jpg
Processing person_0_6.jpg ...
Opening  Datasets/espe/base/validation\0\001_9 28.jpg
Saving file on Datasets/espe/lbp_histograms - copia

In [1]:
from processing import generate_masks_datasets

parent_root = 'Datasets/Propio/primer_grupo/test_data'
target_root = 'Datasets/espe/masks/test'


generate_masks_datasets(parent_root, target_root)

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead
Working on:  Datasets/Propio/primer_grupo/test_data\0
Processing person_0_1.jpg ...
Opening  Datasets/Propio/primer_grupo/test_data\0\person_0_241.jpg
Saving file on Datasets/espe/masks/test\0\person_0_1.jpg
Processing person_0_2.jpg ...
Opening  Datasets/Propio/primer_grupo/test_data\0\person_0_242.jpg
Saving file on Datasets/espe/masks/test\0\person_0_2.jpg
Processing person_0_3.jpg ...
Opening  Datasets/Propio/primer_grupo/test_data\0\person_0_243.jpg
Saving file on Datasets/espe/masks/test\0\person_0_3.jpg
Processing person_0_4.jpg ...
Opening  Datasets/Propio/primer_grupo/test_data\0\person_0_244.jpg
Saving file on Datasets/espe/masks/test\0\person_0_4.jpg
Processing person_0_5.jpg ...
Opening  Datasets/Propio/primer_grupo/test_data\0\p